In [1]:
import sqlite3
import json
from datetime import datetime
import time

timeframe = '2011-09'
sql_transaction = []
start_row = 0
cleanup = 1000000


In [2]:

connection = sqlite3.connect('{}.db'.format(timeframe))
c = connection.cursor()

def create_table():
    c.execute("CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)")

def format_data(data):
    data = data.replace('\n',' newlinechar ').replace('\r',' newlinechar ').replace('"',"'")
    return data

def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transaction = []

def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

def acceptable(data):
    if len(data.split(' ')) > 1000 or len(data) < 1:
        return False
    elif len(data) > 32000:
        return False
    elif data == '[deleted]':
        return False
    elif data == '[removed]':
        return False
    else:
        return True

def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False

def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False


In [5]:
if __name__ == '__main__':
    create_table()
    row_counter = 0
    paired_rows = 0

    with open("RC_2011-09", buffering=1000) as f:
        for row in f:
            #print(row)
            #time.sleep(555)
            row_counter += 1

            if row_counter > start_row:
                try:
                    row = json.loads(row)
                    parent_id = row['parent_id'].split('_')[1]
                    body = format_data(row['body'])
                    created_utc = row['created_utc']
                    score = row['score']
                    
                    comment_id = row['id']
                    
                    subreddit = row['subreddit']
                    parent_data = find_parent(parent_id)
                    
                    existing_comment_score = find_existing_score(parent_id)
                    if existing_comment_score:
                        if score > existing_comment_score:
                            if acceptable(body):
                                sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                                
                    else:
                        if acceptable(body):
                            if parent_data:
                                if score >= 2:
                                    sql_insert_has_parent(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                                    paired_rows += 1
                            else:
                                sql_insert_no_parent(comment_id,parent_id,body,subreddit,created_utc,score)
                except Exception as e:
                    print(str(e))
                            
            if row_counter % 100000 == 0:
                print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))

            if row_counter > start_row:
                if row_counter % cleanup == 0:
                    print("Cleanin up!")
                    sql = "DELETE FROM parent_reply WHERE parent IS NULL"
                    c.execute(sql)
                    connection.commit()
                    c.execute("VACUUM")
                    connection.commit()
                

Total Rows Read: 100000, Paired Rows: 6769, Time: 2017-12-22 17:06:49.029003
Total Rows Read: 200000, Paired Rows: 14214, Time: 2017-12-22 17:07:08.777382
Total Rows Read: 300000, Paired Rows: 22671, Time: 2017-12-22 17:07:28.514717
Total Rows Read: 400000, Paired Rows: 31221, Time: 2017-12-22 17:07:48.281074
Total Rows Read: 500000, Paired Rows: 38914, Time: 2017-12-22 17:08:08.327774
Total Rows Read: 600000, Paired Rows: 46200, Time: 2017-12-22 17:08:28.243317
Total Rows Read: 700000, Paired Rows: 54404, Time: 2017-12-22 17:08:48.682424
Total Rows Read: 800000, Paired Rows: 62872, Time: 2017-12-22 17:09:08.846692
Total Rows Read: 900000, Paired Rows: 70596, Time: 2017-12-22 17:09:28.846552
Total Rows Read: 1000000, Paired Rows: 77906, Time: 2017-12-22 17:09:48.911846
Cleanin up!
Total Rows Read: 1100000, Paired Rows: 85743, Time: 2017-12-22 17:10:13.233442
Total Rows Read: 1200000, Paired Rows: 94070, Time: 2017-12-22 17:10:32.812402
Total Rows Read: 1300000, Paired Rows: 101571, Tim

Total Rows Read: 10300000, Paired Rows: 817972, Time: 2017-12-22 17:47:38.435883
Total Rows Read: 10400000, Paired Rows: 825910, Time: 2017-12-22 17:48:03.208698
Total Rows Read: 10500000, Paired Rows: 832912, Time: 2017-12-22 17:48:27.286376
Total Rows Read: 10600000, Paired Rows: 840871, Time: 2017-12-22 17:48:50.382152
Total Rows Read: 10700000, Paired Rows: 849008, Time: 2017-12-22 17:49:12.675255
Total Rows Read: 10800000, Paired Rows: 856673, Time: 2017-12-22 17:49:37.693490
Total Rows Read: 10900000, Paired Rows: 864501, Time: 2017-12-22 17:50:02.584027
Total Rows Read: 11000000, Paired Rows: 871844, Time: 2017-12-22 17:50:27.218774
Cleanin up!
Total Rows Read: 11100000, Paired Rows: 879618, Time: 2017-12-22 17:51:07.619461
Total Rows Read: 11200000, Paired Rows: 888188, Time: 2017-12-22 17:51:29.519211
Total Rows Read: 11300000, Paired Rows: 896270, Time: 2017-12-22 17:51:51.685435
Total Rows Read: 11400000, Paired Rows: 903760, Time: 2017-12-22 17:52:15.283488
Total Rows Read: